In [ ]:

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
# import tensorflow_probability as tfp
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras import initializers

from tensorflow.keras.layers import Embedding, Input, Dense, Flatten, concatenate, dot, Add, Dropout, BatchNormalization, \
    Activation, LeakyReLU, Lambda, Multiply,Conv1D,Softmax,Conv2D

# import tensorflow_probability as tfp
from tensorflow.keras.optimizers import Adam, Adamax, Adagrad, Adadelta, SGD, Nadam
from tensorflow.keras.layers import Conv1D as cnn1

import numpy as np

from tensorflow.keras.models import Model
import tensorflow as tf
gpu_list = tf.config.experimental.list_physical_devices('GPU')
if len(gpu_list) > 0 :
    for gpu in gpu_list:
        try:
        # 设置多张 GPU ，如果不需要 for 去设置多张，则使用 list 的索引设置即可
            tf.config.experimental.set_memory_growth(gpu, True)
        except RuntimeError as e:
            print(e)
else:
    print("Got no GPUs")
import pandas as pd
from tensorflow.python.keras.engine.base_layer import Layer

from tensorflow.keras import backend as K

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate, Multiply
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import MultiHeadAttention

EMBED_DIM = 512  # Latent dimension

# Embedding helper
def get_embedding(input_dim, output_dim, input_length, name):
    return Embedding(
        input_dim=input_dim,
        output_dim=output_dim,
        input_length=input_length,
        embeddings_initializer=RandomNormal(),
        name=name
    )

# Multi-head self-attention block
def apply_attention(inputs):
    attn = MultiHeadAttention(num_heads=2, key_dim=EMBED_DIM)(inputs, inputs)
    return Dense(EMBED_DIM, activation='relu')(attn)

# Main R2SL model
def create_r2sl_model():
    # Input definition
    inputs = {
        'user_id': Input(shape=(1,), dtype='int64', name='user_id_input'),
        'user_region': Input(shape=(2,), dtype='int64', name='user_lc_input'),
        'item_id': Input(shape=(1,), dtype='int64', name='item_id_input'),
        'item_region': Input(shape=(2,), dtype='int64', name='item_lc_input'),
        'user_id_2': Input(shape=(1,), dtype='int64', name='user_id_2_input'),
        'user_region_2': Input(shape=(2,), dtype='int64', name='user_lc_2_input'),
        'item_id_2': Input(shape=(1,), dtype='int64', name='item_id_2_input'),
        'item_region_2': Input(shape=(2,), dtype='int64', name='item_lc_2_input'),
    }

    # Embedding layers (shared across related inputs)
    emb_user_id = get_embedding(339, EMBED_DIM, 1, 'user_id_embedding')
    emb_user_reg = get_embedding(339, EMBED_DIM, 2, 'user_region_embedding')
    emb_item_id = get_embedding(5825, EMBED_DIM, 1, 'item_id_embedding')
    emb_item_reg = get_embedding(5825, EMBED_DIM, 2, 'item_region_embedding')

    # Apply embedding + flatten
    def embed_and_flatten(layer, name):
        return Flatten(name=name+'_flat')(layer)

    # Embedding lookups
    user_id_latent = embed_and_flatten(emb_user_id(inputs['user_id']), 'user_id')
    user_reg_latent = embed_and_flatten(emb_user_reg(inputs['user_region']), 'user_region')
    item_id_latent = embed_and_flatten(emb_item_id(inputs['item_id']), 'item_id')
    item_reg_latent = embed_and_flatten(emb_item_reg(inputs['item_region']), 'item_region')

    user_id_2_latent = embed_and_flatten(emb_user_id(inputs['user_id_2']), 'user_id_2')
    user_reg_2_latent = embed_and_flatten(emb_user_reg(inputs['user_region_2']), 'user_region_2')
    item_id_2_latent = embed_and_flatten(emb_item_id(inputs['item_id_2']), 'item_id_2')
    item_reg_2_latent = embed_and_flatten(emb_item_reg(inputs['item_region_2']), 'item_region_2')

    # Feature concatenation
    f1 = Concatenate()([user_id_latent, user_reg_latent, item_id_latent, item_reg_latent])
    f2 = Concatenate()([user_id_2_latent, user_reg_2_latent, item_id_2_latent, item_reg_2_latent])
    region_as = Concatenate()([user_reg_latent, user_reg_2_latent])
    region_city = Concatenate()([item_reg_latent, item_reg_2_latent])

    # Shared towers
    def dense_tower(x, name_prefix):
        x = Dense(512, activation='relu', name=f'{name_prefix}_1')(x)
        x = Dense(256, activation='relu', name=f'{name_prefix}_2')(x)
        return Dense(128, activation='relu', name=f'{name_prefix}_3')(x)

    tower_f1 = dense_tower(f1, 'tower_f1')
    tower_f2 = dense_tower(f2, 'tower_f2')
    tower_as = dense_tower(region_as, 'tower_as')
    tower_city = dense_tower(region_city, 'tower_city')

    # Gating: user_id_latent and user_id_2_latent control two gate flows
    def gated_merge(domain_vec, expert_vec, name_prefix):
        g = Dense(512, activation='relu', name=f'{name_prefix}_gate_1')(domain_vec)
        g = Dense(256, activation='relu', name=f'{name_prefix}_gate_2')(g)
        g = Dense(1, activation='sigmoid', name=f'{name_prefix}_gate_sigmoid')(g)
        return Multiply(name=f'{name_prefix}_gated')([expert_vec, g])

    merged1 = gated_merge(user_id_latent, tower_as, 'merge1')
    merged2 = gated_merge(user_id_latent, tower_city, 'merge2')
    merged3 = gated_merge(user_id_2_latent, tower_as, 'merge3')
    merged4 = gated_merge(user_id_2_latent, tower_city, 'merge4')

    # Attention-enhanced fusion
    fusion_rt = Concatenate(name='fusion_rt')([tower_f1, merged1, merged2])
    fusion_tp = Concatenate(name='fusion_tp')([tower_f2, merged3, merged4])

    attn_rt = apply_attention(tf.expand_dims(fusion_rt, axis=1))
    attn_tp = apply_attention(tf.expand_dims(fusion_tp, axis=1))

    # Prediction heads
    def output_head(x, base, task_name):
        x = Dense(2048, activation='relu', name=f'{task_name}_fc1')(x)
        x = Dense(1024, activation='relu', name=f'{task_name}_fc2')(Concatenate()([x, base]))
        x = Dense(512, activation='relu', name=f'{task_name}_fc3')(x)
        return Dense(1, activation='linear', name=task_name)(x)

    rt_output = output_head(attn_rt, tower_f1, 'rt_output')
    tp_output = output_head(attn_tp, tower_f2, 'tp_output')

    model = Model(inputs=list(inputs.values()), outputs=[rt_output, tp_output])
    return model

# Instantiate and compile the model
model = create_r2sl_model()
model.compile(optimizer='adam',
              loss={'rt_output': 'mae', 'tp_output': 'mae'},
              metrics={'rt_output': 'mae', 'tp_output': 'mae'})
model.summary()


In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

df_rt = pd.read_csv('./Data/WSDream/Dataset#1/rt_train_0.075.txt', sep='\t')
df_tp = pd.read_csv('./Data/WSDream/Dataset#1/tp_train_0.075.txt', sep='\t')
df_tp = df_tp.rename(columns={col: col + "_2" if col != "TP" else col for col in df_tp.columns})


df = pd.concat([df_rt, df_tp], axis=1)
df = df.dropna()


rt_labels = df['[RT]'].values
tp_labels = df['[TP]_2'].values
x_train=[df[['[User ID]']],df[['[User AS]','[User Country]']],df[['[Service ID]']],df[['[Service AS]','[Service Country]']],df[['[User ID]_2']],df[['[User AS]_2','[User Country]_2']],df[['[Service ID]_2']],df[['[Service AS]_2','[Service Country]_2']]]


df_test_rt = pd.read_csv('./Data/WSDream/Dataset#1/rt_test_0.075.txt', sep='\t')
df_test_tp = pd.read_csv('./Data/WSDream/Dataset#1/tp_test_0.075.txt', sep='\t')
df_test_tp = df_test_tp.rename(columns={col: col + "_2" if col != "TP" else col for col in df_test_tp.columns})


df_test = pd.concat([df_test_rt, df_test_tp], axis=1)
df_test = df_test.dropna()


rt_test_labels = df_test['[RT]'].values
tp_test_labels = df_test['[TP]_2'].values

x_test=[df_test[['[User ID]']],df_test[['[User AS]','[User Country]']],df_test[['[Service ID]']],df_test[['[Service AS]','[Service Country]']],df_test[['[User ID]_2']],df_test[['[User AS]_2','[User Country]_2']],df_test[['[Service ID]_2']],df_test[['[Service AS]_2','[Service Country]_2']]]


# compile autoencoder
# autoencoder.compile(optimizer='adam', loss=huber_loss,metrics=['mae'])
# from Evaluator import evaluate, saveResult

autoencoder = create_multi_task_model()
autoencoder.compile(optimizer='Nadam', 
              loss={'rt_output': 'mae', 'tp_output': 'mae'}, 
        #        loss={'rt_output': 'mae'}, 
              loss_weights={'rt_output': 1., 'tp_output':1.},
              metrics={'rt_output': 'mae', 'tp_output': 'mae'})
import sys
from time import time
evalResults = np.zeros((30, 3))
sys.stdout.write('\rInitializing...')
rt_test, tp_rest = autoencoder.predict(x_test, batch_size=64)
mae = mean_absolute_error(rt_test_labels, rt_test)
rmse = np.sqrt(mean_squared_error(rt_test_labels, rt_test))
mae_tp = mean_absolute_error(tp_test_labels, tp_rest)
rmse_tp = np.sqrt(mean_squared_error(tp_test_labels, tp_rest))

sys.stdout.write('\rInitializing completes.MAE = %.4f|RMSE = %.4f.\n' % (mae, rmse))
best_mae, best_rmse, best_epoch = mae, rmse, -1
for epoch in range(30):
        sys.stdout.write('\rEpoch %d starts...' % epoch)
        start = time()
        # Training
        history = autoencoder.fit(x_train, {'rt_output': rt_labels, 'tp_output': tp_labels}, batch_size=64, epochs=1, verbose=0, shuffle=True)
        # , callbacks=[TensorBoard(log_dir='./Log')])
        end = time()
        sys.stdout.write('\rEpoch %d ends.[%.1fs]' % (epoch, end - start))
        # print(end - start)
        # Evaluation
        if epoch % 1 == 0:
                sys.stdout.write('\rEvaluating Epoch %d...' % epoch)
                print("\n")
                if epoch>0:
                        rt_test, tp_rest = autoencoder.predict(x_test, batch_size=64)
                        mae = mean_absolute_error(rt_test_labels, rt_test)
                        rmse = np.sqrt(mean_squared_error(rt_test_labels, rt_test))
                        mae_tp = mean_absolute_error(tp_test_labels, tp_rest)
                        rmse_tp = np.sqrt(mean_squared_error(tp_test_labels, tp_rest))
                        loss = history.history['loss'][0]
                        sys.stdout.write('\rEvaluating completes.[%.1fs] ' % (time() - end))
                        if mae + rmse < best_mae + best_rmse:
                                best_mae, best_rmse, best_epoch = mae, rmse, epoch
                        evalResults[epoch, :] = [mae, rmse,loss]
                        sys.stdout.write('\rEpoch %d : MAE = %.4f|RMSE = %.4f|MAE_tp = %.4f|RMSE_tp = %.4f|Loss = %.4f\n' % (epoch, mae, rmse,mae_tp,rmse_tp, loss))
print('=' * 14 + 'Training Complete!' + '=' * 18)
print('The best is at epoch %d : MAE = %.4f|RMSE = %.4f.' % (best_epoch, best_mae, best_rmse))